In [100]:
import pandas as pd
import glob
import re
#警告を無視するために導入（警告の中身を確認してから使うか否かを判断するのが吉）
import warnings
warnings.simplefilter('ignore')

In [101]:
def search_cell(data, target_str):
    '''
    特定の文字列の場所を取得して辞書にする関数。
    <input>
        data       : 対象のデータフレーム(DF)
        target_str : 探したい文字列(str)
    <output>
        ind_col_dic : 探したい文字列があるセル番地(dict)
    '''
    col_name = data.columns

    #品名のセルを見つける
    for c in range(len(col_name)):
        try:
            target = data[data[col_name[c]].isin([target_str])]
            if len(target) != 0:
                #何列目かを記憶させておく
                col = c
                #何行目かを記憶させておく
                ind = target.index[0]
                #辞書として行・列を格納
                keys = ['index', 'column']
                values = [ind, col]
                ind_col_dic = dict(zip(keys, values))
        except:
            continue

    return ind_col_dic


#ファイル読み込みからデータフレーム成型まで一連の処理を関数にする。
def make_df(path):
    '''
    見積書を読込み、データフレームを成型する。
    <input>
        path : 読込みたいエクセルファイルのパス(str)
    <output>
        df_summary : 必要事項のみを抽出したデータフレーム(DataFrame)
    '''
    data = pd.read_excel(path)
    # data = pd.read_excel(path, engine="openpyxl")

    #不要な半角空白を除去
    data = data.replace('\s', '', regex=True)
    #不要な全角空白を除去（もしかしたら全角空白もあるかもしれない）
    data = data.replace('　', '', regex=True)
    #※必要であると感じたら、\tや\nの除去も追加しましょう。

    #「品名」のセルを見つける
    cell_partsname = search_cell(data, '品名')
    #「様」のセルを見つける
    cell_sama = search_cell(data, '様')

    idx_partsname = cell_partsname['index']
    col_partsname = cell_partsname['column']
    data1 = data.iloc[idx_partsname:, col_partsname:col_partsname+5]
    data2 = data1.reset_index(drop=True)
    nan_idx = data2[data2.iloc[:, 0].isnull()==True].index.tolist()
    data3 = data2.iloc[:nan_idx[0], :]


    #カラムに入れたい0番目の値をリストに格納し、カラムとして定義する。
    cols = data3.iloc[0, :].tolist()
    data3.columns = cols

    #0番目の行は不要なのでスライスで除去
    data4 = data3.iloc[1:, :]

    #納入先の取得
    supplier = data.iloc[cell_sama['index'], cell_sama['column']-1]

    #購入日の取得
    data_str = data.iloc[:3, :].dropna(how='all', axis=1).dropna(how='all', axis=0).astype(str)
    df_str_l = data_str.values.tolist()[0]
    #-を含み、かつ数字から始まるものを抽出
    buy_date = [s for s in df_str_l if ('-' in s) and (re.match('[0-9]' , s))][0]
    # buy_date = data.iloc[cell_sama['index'], cell_sama['column']]

    #データフレームへの追加
    data4['購入日'] = buy_date
    data4['納入先'] = supplier

    #並べ替えの方法はいくつかあるが、ここでは購入日より右、左でデータフレームを分割し、再度結合する方法をとる。
    data4_r = data4.iloc[:, 5:]
    data4_l = data4.iloc[:, :5]

    #データフレームの結合
    data5 = pd.concat([data4_r, data4_l], axis=1)

    return data5

In [102]:
#exampleフォルダ配下の最下層にあるエクセルファイルのパスをすべて取得
file_list = glob.glob('example/*/*.xl*')

#すべてのファイルから必要なものを抽出してDFに成形
for i in range(len(file_list)):
    if i == 0:
        df = make_df(file_list[0])
    else:
        dfx = make_df(file_list[i])
        df = pd.concat([df, dfx], axis=0)
df

,購入日,納入先,品名,単価,数量,値引き額,金額
1,2018-01-10,株式会社〇×▽,トランジスタ,50,50,0,2500
2,2018-01-10,株式会社〇×▽,IC,100,5,50,450
3,2018-01-10,株式会社〇×▽,抵抗器,5,10000,1000,49000
1,2019-01-20,株式会社〇×▽,トランジスタ,50,100,100,4900
2,2019-01-20,株式会社〇×▽,IC,100,10,50,950
1,2020-01-19,株式会社〇×▽,抵抗器,5,10000,1000,49000
2,2020-01-19,株式会社〇×▽,ＬＥＤ,30,1000,500,29500
3,2020-01-19,株式会社〇×▽,コンデンサ,20,1000,100,19900
1,2021-02-01,株式会社●●●,トランジスタ,50,100,0,5000
2,2021-02-01,株式会社●●●,IC,100,20,10,1990
